In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from random import randint
from collections import Counter


In [32]:
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
print(type(mnist))

<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>


In [0]:
X, y, k = mnist.test.images, mnist.test.labels, 10

In [35]:
print(type(X), type(y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [0]:
# select random points as a starting position. You can do better by randomly selecting k points.
start_pos = tf.Variable(X[np.random.randint(X.shape[0], size=k),:], dtype=tf.float32)
centroids = tf.Variable(start_pos.initialized_value(), 'S', dtype=tf.float32)

In [0]:
# populate points
points           = tf.Variable(X, 'X', dtype=tf.float32)
ones_like        = tf.ones((points.get_shape()[0], 1))
prev_assignments = tf.Variable(tf.zeros((points.get_shape()[0], ), dtype=tf.int64))

In [0]:
# find the distance between all points: http://stackoverflow.com/a/43839605/1090562
p1 = tf.matmul(
    tf.expand_dims(tf.reduce_sum(tf.square(points), 1), 1),
    tf.ones(shape=(1, k))
)
p2 = tf.transpose(tf.matmul(
    tf.reshape(tf.reduce_sum(tf.square(centroids), 1), shape=[-1, 1]),
    ones_like,
    transpose_b=True
))
distance = tf.sqrt(tf.add(p1, p2) - 2 * tf.matmul(points, centroids, transpose_b=True))


In [0]:
# assign each point to a closest centroid
point_to_centroid_assignment = tf.argmin(distance, axis=1)

In [0]:
# recalculate the centers
total = tf.unsorted_segment_sum(points, point_to_centroid_assignment, k)
count = tf.unsorted_segment_sum(ones_like, point_to_centroid_assignment, k)
means = total / count

In [0]:
# continue if there is any difference between the current and previous assignment
is_continue = tf.reduce_any(tf.not_equal(point_to_centroid_assignment, prev_assignments))

In [0]:
with tf.control_dependencies([is_continue]):
    loop = tf.group(centroids.assign(means), prev_assignments.assign(point_to_centroid_assignment))

sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [0]:
# do many iterations. Hopefully you will stop because of has_changed is False
has_changed, cnt = True, 0
while has_changed and cnt < 300:
    cnt += 1
    has_changed, _ = sess.run([is_continue, loop])

In [0]:
# see how the data is assigned
res = sess.run(point_to_centroid_assignment)

In [48]:
nums_in_clusters = [[] for i in range(10)]
for cluster, real_num in zip(list(res), list(y)):
    nums_in_clusters[cluster].append(real_num)

for i in range(10):
    print (Counter(nums_in_clusters[i]).most_common(3))

[(0, 753), (6, 19), (2, 13)]
[(6, 651), (4, 26), (0, 16)]
[(3, 708), (5, 292), (8, 108)]
[(1, 483), (2, 121), (7, 54)]
[(8, 630), (5, 166), (3, 133)]
[(7, 647), (9, 405), (4, 292)]
[(1, 645), (3, 52), (2, 51)]
[(5, 284), (6, 240), (0, 145)]
[(2, 682), (3, 35), (7, 15)]
[(4, 549), (9, 528), (7, 265)]
